In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from networkx.readwrite import json_graph

import os, json

with open("data/cisplatin_pos_kg.json") as f:
    data = json.load(f)

G = json_graph.node_link_graph(data, edges="edges")
print(G)

Graph with 50113 nodes and 674225 edges


In [2]:
# export to cytoscape for visualization
cyto_data = nx.readwrite.json_graph.cytoscape_data(G)
with open("data/cisplatin_pos_kg_cyto.json", "w") as f:
    json.dump(cyto_data, f)